Подключимся к гугл-диску, чтобы получить доступ к датасету:

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Установим нужные нам версии библиотек:



In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install keras==2.2.2
!pip install tensorflow==1.15.0
!pip install keras_applications==1.0.7








Задаем пути 

In [0]:
time_dir = '/content/gdrive/My Drive/Mezentseva Zavarzina/experiments_dl_model/'
# имя папки, где будут результаты оценки и часть имени с весами
path_to_exp = '120_5_3_150_chars_elmo_2'
result_dir = '/content/gdrive/My Drive/Mezentseva Zavarzina/experiments_dl_model/test_ml/'

In [0]:
import csv

import os
import zipfile
import numpy as np
from tqdm import tqdm

import tensorflow as tf
import tensorflow_hub as hub
import keras
from keras import backend as K
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input
from keras.layers.merge import add, concatenate
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras_contrib.layers import CRF

Using TensorFlow backend.


Распакуем архив с данными:

In [0]:
if 'train' in os.listdir(time_dir):
  print('Files are already extracted')
else:
  with zipfile.ZipFile(os.path.join(time_dir, 'train.zip'), 'r') as zip_ref:
      zip_ref.extractall(os.path.join(time_dir, 'train'))

print(len(os.listdir(os.path.join(time_dir, 'train'))))

FileNotFoundError: ignored

Импортируем нужные нам библиотеки:

In [0]:
print(tf.__version__)
print(keras.__version__)

1.15.0
2.2.2


Определим класс для загрузки наших данных:

In [0]:
class DatasetLoader:

    def __init__(self, path_to_data_dir):
        self._path_to_data_dir = path_to_data_dir
        self._data_files = sorted(os.listdir(self._path_to_data_dir))


    def load_dataset(self):
        sentences = []
        for data_file in tqdm(self._data_files, desc='Loading data'):
            if not data_file.endswith('.csv'):
              continue
            with open(os.path.join(self._path_to_data_dir, data_file), 'r') as data_f:
                reader = csv.DictReader(data_f)
                sentence = []
                for row in reader:
                    sentence.append((row['token'], row['tag']))
                sentences.append(sentence)
        return sentences


Определим класс для векторизации данных:

In [0]:
class Vectorizer:

    def __init__(self):
        self._max_sentence_len = 1000
        self._max_wordform_len = 30
        self._all_chars = u'qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM,.?!:;"«»-—1234567890'
        self._punct = u',.?!:;"«»-—'
        self._letters = u'qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM'
        self._numbers = u'1234567890'
        self._len_all_chars = len(self._letters) + 2
        
        self._tag2id = {'O': 0, 'B-TIME': 1, 'I-TIME': 2, 'B-DATE': 3, 'I-DATE': 4, 'B-DURATION': 5, 'I-DURATION': 6,
                        'B-SET': 7, 'I-SET': 8}
    
    def get_len_all_chars(self):
      return self._len_all_chars

    def vectorize_chars_and_tokens(self, sentences):
        X_chars = self.vectorize_chars_dataset(sentences)
        X_sentences = self.vectorize_sentences(sentences)
        return X_chars, X_sentences

    def vectorize_targets(self, sentences):
        y = [[self._tag2id[w[1]] for w in s] for s in sentences]
        y = pad_sequences(maxlen=self._max_sentence_len, sequences=y, padding="post", value=self._tag2id["O"])
        y = y.reshape(y.shape[0], y.shape[1], 1)
        return y

    def vectorize_chars_dataset(self, sentences):
        X_chars = [[self._vectorize_chars_wordform(w[0]) for w in s] for s in sentences]
        X_chars = pad_sequences(maxlen=self._max_sentence_len, sequences=X_chars, padding="post",
                                value=np.zeros((self._max_wordform_len, self._len_all_chars), dtype=np.int32))
        return X_chars

    def _vectorize_chars_wordform(self, wordform):
        vector = np.zeros(self._max_wordform_len * self._len_all_chars)
        for i in range(len(wordform)):
            if i == self._max_wordform_len:
                break
            """
            if wordform[i] in self._all_chars:
                ind = self._all_chars.index(wordform[i])
                vector[i * self._len_all_chars + ind] = 1.0
            """
            # try to make 3 classes
            if wordform[i] in self._punct:
              ind = len(self._letters) + 1
              vector[i * self._len_all_chars + ind] = 1.0              
            if wordform[i] in self._letters:
              ind = self._letters.index(wordform[i])
              vector[i * self._len_all_chars + ind] = 1.0
            if wordform[i] in self._numbers:  
              ind = len(self._letters) + 2
              vector[i * self._len_all_chars + ind] = 1.0
        vector = vector.reshape((self._max_wordform_len, self._len_all_chars))
        return vector

    def vectorize_sentences(self, sentences):
        X = [[w[0] for w in s] for s in sentences]
        new_X = []
        for seq in X:
            new_seq = []
            for i in range(self._max_sentence_len):
                try:
                    new_seq.append(seq[i])
                except:
                    new_seq.append("__PAD__")
            new_X.append(new_seq)
        X = new_X
        return np.array(X)

Определим класс для обучения модели:

In [0]:
class Trainer:

    def __init__(self):
        data_dir = os.path.join(time_dir, 'train')
        self._data_loader = DatasetLoader(data_dir)
        self._vectorizer = Vectorizer()

        self._sentences = self._data_loader.load_dataset()
        self._val_sentences = self._sentences[-50:]
        self._train_samples = self._sentences[:200]

        self._batch_size = 5
        self._max_len = 1000
        self.steps_per_epoch = len(self._train_samples)/self._batch_size

        sess = tf.Session()
        K.set_session(sess)

        print('Loading elmo model...')
        self._elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
        print('Elmo model is loaded')
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())


        self._model = self._define_model()
      
    def ElmoEmbedding(self, x):
        return self._elmo_model(inputs={
            "tokens": tf.squeeze(tf.cast(x, tf.string)),
            "sequence_len": tf.constant(self._batch_size * [self._max_len])
        },
            signature="tokens",
            as_dict=True)["elmo"]

    def _define_model(self):
        input_chars = Input(shape=(self._max_len, 30, self._vectorizer.get_len_all_chars()))
        chars = TimeDistributed(Bidirectional(LSTM(units=150,
                                                   recurrent_dropout=0.5)))(input_chars)
  
        input_text = Input(shape=(self._max_len,), dtype="string")
        embedding = Lambda(self.ElmoEmbedding, output_shape=(self._max_len, 1024))(input_text)
        x = Bidirectional(LSTM(units=512, return_sequences=True,
                               recurrent_dropout=0.2, dropout=0.2))(embedding)
        x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
                                   recurrent_dropout=0.2, dropout=0.2))(x)

        x = concatenate([x_rnn, chars])  # residual connection to the first biLSTM
        crf = CRF(9, sparse_target=True)  # CRF layer
        out = crf(x)  # output

        model = Model([input_chars, input_text], out)
        model.summary()

        model.compile(optimizer="adam", loss=crf.loss_function, metrics=[crf.accuracy])
        return model

    def _generate_train_samples(self):
      i = 0
      while True:
        X_chars, X_sentences = self._vectorizer.vectorize_chars_and_tokens(self._train_samples[i:i + self._batch_size])
        y = self._vectorizer.vectorize_targets(self._train_samples[i:i + self._batch_size])
        i += self._batch_size
        yield [np.array(X_chars), np.array(X_sentences)], np.array(y)
        if i == len(self._train_samples):
          i = 0

    def _generate_val_samples(self):
      i = 0
      while True:
        X_chars, X_sentences = self._vectorizer.vectorize_chars_and_tokens(self._val_sentences[i:i + self._batch_size])
        y = self._vectorizer.vectorize_targets(self._val_sentences[i:i + self._batch_size])
        i += self._batch_size
        yield [np.array(X_chars), np.array(X_sentences)], np.array(y)
        if i == len(self._val_sentences):
          i = 0

    def train(self):
        weights_dir = '/content/gdrive/My Drive/Mezentseva Zavarzina/experiments_dl_model/weights_nastya'
        if not os.path.exists(weights_dir):
          os.makedirs(weights_dir)
        weights_file = 'weights_chars_crf_' + path_to_exp + '_.h5'
        modelPath = os.path.join(weights_dir, weights_file)
        saver = ModelCheckpoint(modelPath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
        stopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
        history = self._model.fit_generator(self._generate_train_samples(), epochs=120,
                            validation_data=self._generate_val_samples(), steps_per_epoch=self.steps_per_epoch, validation_steps=10,
                            verbose=1, callbacks=[saver, stopper])
        return history

In [0]:
trainer = Trainer()
trainer.train()

Loading data: 100%|██████████| 256/256 [03:20<00:00,  1.28it/s]


Loading elmo model...
Elmo model is loaded



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/120



40/40 [==============================] - 347s 9s/step - loss: 0.1745 - crf_viterbi_accuracy: 0.9619 - val_loss: 0.0831 - val_crf_viterbi_accuracy: 0.9813

Epoch 00001: val_loss improved from inf to 0.08307, saving model to /content/gdrive/My Drive/Mezentseva Zavarzina/experiments_dl_model/weights_nastya/weights_chars_crf_120_5_3_150_chars_elmo_2_.h5
Epoch 2/120
40/40 [==============================] - 333s 8s/step - loss: 0.0554 - crf_viterbi_accuracy: 0.9873 - val_loss: 0.0550 - val_crf_viterbi_accuracy: 0.9866

Epoch 00002: val_loss improved from 0.08307 to 0.05501, saving model to /content/gdrive/My Drive/Mezentseva Zavarzina/experiments_dl_model/weights_nastya/weights_chars_crf_120_5_3_150_chars_elmo_2_.h5
Epoch 3/120
40/40 [==============================] - 333s 8s/step - loss: 0.0423 - crf_viterbi_accuracy: 0.9894 - val_loss: 0.0462 - val_crf_viterbi_accuracy: 0.9877

Epoch 00003: val_loss improved from 0.05501 to 0.04621, saving model to /content/gdrive/My Drive/Mezentseva Zavar

Теперь давайте попробуем найти временные выражения, используя обученную модель. Для этого напишем класс **Predictor**

In [0]:
from nltk.tokenize import WordPunctTokenizer


class Predictor:

    def __init__(self):
        self._vectorizer = Vectorizer()
        self._trainer = Trainer()
        self._tokenizer = WordPunctTokenizer()
        self._model = self._trainer._model
        self._model.load_weights('/content/gdrive/My Drive/Mezentseva Zavarzina/experiments_dl_model/weights_nastya/weights_chars_crf_' + path_to_exp + '_.h5')
        self._tag2id = {'O': 0, 'B-TIME': 1, 'I-TIME': 2, 'B-DATE': 3, 'I-DATE': 4, 'B-DURATION': 5, 'I-DURATION': 6,
                        'B-SET': 7, 'I-SET': 8}
        self._id2tag = self._get_id2tag()

    def _get_id2tag(self):
        id2tag = dict()
        for tag, id in self._tag2id.items():
            id2tag[id] = tag
        return id2tag

    def _process_input_sentence(self, sentence):
        tokens = self._tokenizer.tokenize(sentence)
        tokens_to_process = [(token, 'O') for token in tokens]
        input_tokens = []
        for i in range(5):
            input_tokens.append(tokens_to_process)
        X_chars, X_sequence = self._vectorizer.vectorize_chars_and_tokens(sentences=input_tokens)
        return X_chars, X_sequence, tokens

    def predict(self, text):
        X_chars, X_sequence, tokens = self._process_input_sentence(text)
        predicts = self._model.predict([X_chars, X_sequence])[0]
        result = []
        for i, token in enumerate(tokens):
            tag = self._id2tag[np.argmax(predicts[i])]
            result.append((token, tag))
        return result


In [0]:
predictor = Predictor()

Loading data: 100%|██████████| 256/256 [00:00<00:00, 412.43it/s]


Loading elmo model...
Elmo model is loaded
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 1000, 1024)   0           input_6[0][0]                    
__________________________________________________________________________________________________
bidirectional_8 (Bidirectional) (None, 1000, 1024)   6295552     lambda_3[0][0]                   
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 1000, 30, 54) 0                                            
__________________________________________________________________________________________________
bidirectio

/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [0]:
text = 'he was born in 1994'
result = predictor.predict(text)
for r in result:
    print(r)

('he', 'O')
('was', 'O')
('born', 'O')
('in', 'O')
('1994', 'B-DATE')


Определим класс для оценки качества извлечения временных выражений

In [0]:
!pip install seqeval
from seqeval.metrics import classification_report

     |████████████████████████████████| 378kB 6.7MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=5a7ff659ce1d72e1614cce0328abfee30baa7b3b3957eab2502a929a3e31147d
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval
  Found existing installation: Keras 2.2.2
    Uninstalling Keras-2.2.2:
      Successfully uninstalled Keras-2.2.2


In [0]:
class Evaluator:
    """ Класс для оценки качества извлечения временных выражений """

    def __init__(self, file_dir, column_pred):
        self._predicted_files_dir = os.path.join('/content/gdrive/My Drive/Mezentseva Zavarzina/experiments_dl_model/test_ml/', path_to_exp)
        self._predicted_files = sorted(os.listdir(self._predicted_files_dir))
        self.column_pred = column_pred

    def evaluate(self) -> str:
        """ Оценка качества извлечения временных выражений
        :return: метрики: precision, recall, F-1 score для каждого класса отдельно и для всех усреднённые
        """
        preds, targets = self._load_predictons()
        report = classification_report(y_true=targets, y_pred=preds)
        return report

    def _load_predictons(self):
        """ Загружает targets и predictions из заранее сформированных csv файлов
        :return: predictions, targets
        """
        preds = []
        targets = []
        for predicted_file in self._predicted_files:
            with open(os.path.join(self._predicted_files_dir, predicted_file), 'r') as predicted_file:
                reader = csv.DictReader(predicted_file)
                preds_from_file = []
                targets_from_file = []
                for row in reader:
                    preds_from_file.append(row[str(self.column_pred)])
                    targets_from_file.append(row['tag'])
                preds.append(preds_from_file)
                targets.append(targets_from_file)
        return preds, targets


Разметка тестового датасета,  получение метрик

In [0]:
class Labeler:

  def __init__(self, path_to_data_dir):
        self._path_to_data_dir = path_to_data_dir
        self._data_files = sorted(os.listdir(self._path_to_data_dir))

 
 
  def label_dataset(self, column_name):
        """
        функция, которая записывает новую информацию в датасет
        :param column_name: имя столбца
        """
        for data_file in tqdm(self._data_files, desc='Adding data'):
            path_to_test_file = os.path.join(self._path_to_data_dir, data_file)
            print(1)
            with open(path_to_test_file, 'r') as data_f:
                reader = csv.DictReader(data_f)
                print(2)
                test_dir = result_dir + path_to_exp
                with open(os.path.join(test_dir, data_file), 'w', encoding='utf-8', newline='') as task:
                    print('ok')                  
                    fieldnames = ['token', 'tag', column_name]
                    writer = csv.DictWriter(task, fieldnames=fieldnames)
                    writer.writeheader()
                    predictions = list() 
                    data = list()
                    sentence = list()
                    for row in reader:
                      data.append((row['token'], row['tag']))
                      sentence.append(row['token'])
                    predictions = predictor.predict(' '.join(sentence))
                    for i, elem in enumerate(predictions):
                      token, tag = data[i]
                      token, pred_tag = elem 
                      writer.writerow({'token' : token, 'tag' : tag, column_name: pred_tag})

In [0]:
ml_label = Labeler(os.path.join(time_dir, 'test'))
tag = 'predicted_ml_tag'
ml_label.label_dataset(tag)
evaluator = Evaluator(os.path.join(time_dir, 'test'), tag)
report = evaluator.evaluate()
print(report)


Adding data:   0%|          | 0/20 [00:00<?, ?it/s]

1
2
ok



Adding data:   5%|▌         | 1/20 [00:06<01:57,  6.20s/it]

1
2
ok



Adding data:  10%|█         | 2/20 [00:12<01:51,  6.17s/it]

...

1
2
ok



Adding data:  95%|█████████▌| 19/20 [01:56<00:06,  6.13s/it]

1
2
ok



Adding data: 100%|██████████| 20/20 [02:02<00:00,  6.13s/it]

           precision    recall  f1-score   support

     DATE       0.83      0.68      0.75       101
 DURATION       0.41      0.51      0.46        35
      SET       0.00      0.00      0.00         4
     TIME       0.50      0.25      0.33         4

micro avg       0.68      0.61      0.64       144
macro avg       0.70      0.61      0.65       144

